In [1]:
import os
import random
import pickle
import networkx as nx
import math

In [2]:
def load_random_graph(graphs_folder):
    graph_files = [f for f in os.listdir(graphs_folder) if f.endswith('.pkl')]
    random_graph_file = random.choice(graph_files)
    with open(os.path.join(graphs_folder, random_graph_file), 'rb') as f:
        G = pickle.load(f)
    return G, random_graph_file

# Putanja do foldera sa grafovima
graphs_folder = 'grafovi'

# Učitavanje grafa
G_global, graph_file_name = load_random_graph(graphs_folder)
# G_info = nx.info(G)
num_nodes = G_global.number_of_nodes()

# G_info, num_nodes, graph_file_name
num_nodes, graph_file_name

(87, 'graf_25.pkl')

In [3]:
def solution_quality(solution):
    return len(solution.nodes())

In [4]:
def local_search(G, k, current_solution):
    # Ova funkcija treba da smanji broj čvorova u rešenju, ako je to moguće
    # Proverava da li može da ukloni bilo koji čvor a da i dalje ostane k-vezan
    print(f'local_search: Broj cvorova na ulasku u local_search {current_solution.number_of_nodes()}')
    temp_solution = current_solution.copy()
    nodes = list(current_solution.nodes())
    random.shuffle(nodes)
    
    for node in nodes:
        temp_solution.remove_node(node)
        
        if not is_k_connected(temp_solution, k):
            temp_solution.add_node(node)

    temp_solution = G.subgraph(temp_solution.nodes())
    
    print(f'local_search: Broj cvorova na izlasku iz local_search {temp_solution.number_of_nodes()} Broj grana na podgrafa {temp_solution.number_of_edges()}')

    return temp_solution

In [5]:
def is_k_connected(G, k):
    if G.number_of_nodes() < k:
        return False
    else:
        return nx.node_connectivity(G) >= k


In [6]:
def shaking(G, k, current_solution):
    # Dodavanje ili uklanjanje čvorova ili ivica
    modified_solution = current_solution.copy()

    if random.choice([True, False]):
        # Remove randomly selected nodes, but not less than k
        potential_nodes_to_remove = list(modified_solution.nodes())
        # Calculate weights based on the number of neighbors
        weights_to_remove = [G.degree(node) for node in potential_nodes_to_remove]
        # Ensure k is not greater than the number of potential nodes to remove
        k = min(k, len(potential_nodes_to_remove))
        # Choose nodes to remove based on weights
        nodes_to_remove = random.choices(potential_nodes_to_remove, weights=weights_to_remove, k=k)
        modified_solution.remove_nodes_from(nodes_to_remove)
    else:
        # Add randomly selected nodes from the original graph
        potential_nodes = list(set(G.nodes()) - set(modified_solution.nodes()))
        # Calculate weights based on the number of neighbors
        weights = [G.degree(node) for node in potential_nodes]
        
        # Ensure there are nodes available to add
        if potential_nodes:
            # Choose nodes to add based on weights
            nodes_to_add = random.choices(potential_nodes, weights=weights, k=math.floor(len(potential_nodes)/2))
            modified_solution.add_nodes_from(nodes_to_add)
            
            # Add corresponding edges
            for node in nodes_to_add:
                for neighbor in G.neighbors(node):
                    # Ensure that the edge is added only if the neighbor is already in the modified solution
                    if neighbor in modified_solution.nodes():
                        modified_solution.add_edge(node, neighbor)
    
    # Provera da li je i dalje k-vezan
    print(f'shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: {modified_solution.number_of_nodes()}')
    modified_solution = remove_nodes_with_less_than_k_neighbors(modified_solution, k)
    print(f'shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: {modified_solution.number_of_nodes()}')
    if is_k_connected(modified_solution, k):
        print("shaking: modifikovano resenje je prihvaljivo")
        return modified_solution
    else:
        print("shaking: modifikovano resenje nije prihvaljivo")
        return current_solution


In [7]:
def remove_nodes_with_less_than_k_neighbors(G, k):
    changed = True
    while changed:
        changed = False
        nodes_to_remove = [node for node in G.nodes if G.degree(node) < k]
        if nodes_to_remove:
            G.remove_nodes_from(nodes_to_remove)
            changed = True
    return G

In [8]:
from time import perf_counter
import matplotlib.pyplot as plt

In [9]:
def vns_algorithm(G, k, max_iter=5):
    start_time = perf_counter()
    G = remove_nodes_with_less_than_k_neighbors(G, k)
    best_solution = G.copy()
    print(f'vns_algorithm: broj cvorova na pocetku: {G.number_of_nodes()}')
    while perf_counter() - start_time < 30:
        # Shaking
        shaken_solution = shaking(G, k, best_solution)
        # Local search
        new_solution = local_search(G, k, shaken_solution)
        
        # Ažuriranje trenutnog rešenja ako je novo rešenje bolje
        if solution_quality(new_solution) < solution_quality(best_solution):
            best_solution = new_solution.copy()
            print(f'vns_algorithm: nadjeno je novo najbolje resenje: Nodes: {best_solution.number_of_nodes()} Edges:  {best_solution.number_of_edges()}')
    
    return best_solution

In [10]:
# final_solution = vns_algorithm(G_global, 5, 10)
# print("Broj cvorova poslednjeg grafa")
# print(final_solution.number_of_nodes())


In [11]:
# nx.draw(final_solution,with_labels=True, font_weight='bold')
# plt.show()

In [12]:
from itertools import combinations

def brute_force_algorithm(G, k):
    all_nodes = list(G.nodes())
    n = len(all_nodes)
    print("brute_force")
    # Provera svih mogućih kombinacija čvorova od najmanjeg broja čvorova ka većima
    for r in range(k, n+1):
        for nodes in combinations(all_nodes, r):
            subgraph = G.subgraph(nodes)
            if is_k_connected(subgraph, k):
                print(f'brute_force: {subgraph.number_of_nodes()}')                
                return subgraph  # Vraća prvi graf koji zadovoljava uslov

                
    print("brute_force")
    return None

In [13]:
# optimal_solution = brute_force_algorithm(final_solution, 5)
# print(f'brute_force_algorithm: Nodes: {optimal_solution.number_of_nodes()} Edges:  {optimal_solution.number_of_edges()}')
# nx.draw(optimal_solution, with_labels=True, font_weight='bold')
# plt.show()

In [14]:
def ucitaj_grafove_iz_dir(direktorijum='data_sets'):
    ucitani_grafovi = []

    for ime_fajla in os.listdir(direktorijum):
        if ime_fajla.endswith(".pkl"):
            putanja = os.path.join(direktorijum, ime_fajla)
            with open(putanja, 'rb') as file:
                ucitani_graf = pickle.load(file)
                ucitani_grafovi.append(ucitani_graf)

    return ucitani_grafovi

In [15]:
ucitani_grafovi = ucitaj_grafove_iz_dir()

In [16]:
# for i, graf in enumerate(ucitani_grafovi, start=1):
#     print(f"Informacije o grafu {i}:")
#     print(f"  Broj čvorova: {len(graf.nodes)}")
#     print(f"  Broj grana: {len(graf.edges)}")
#     vns_algorithm(graf, 3, 10)
#     print("-------------------")

In [17]:
def ispisi_informacije_o_grafu(indeks, direktorijum='data_sets'):
    ime_fajla = f'graf_{indeks}.pkl'
    putanja = os.path.join(direktorijum, ime_fajla)

    if os.path.exists(putanja):
        with open(putanja, 'rb') as file:
            ucitani_graf = pickle.load(file)
            print(f"Informacije o grafu {indeks}:")
            print(f"  Broj čvorova: {len(ucitani_graf.nodes)}")
            print(f"  Broj grana: {len(ucitani_graf.edges)}")
            final_solution = vns_algorithm(ucitani_graf, 4, 10)
            print(f'Broj cvorova poslednjeg resenja je: {final_solution.number_of_nodes()}')
            brute_force_algorithm(ucitani_graf, 4)
            
    else:
        print(f"Graf {indeks} ne postoji u direktorijumu {direktorijum}.")

In [18]:
ispisi_informacije_o_grafu(1)

Informacije o grafu 1:
  Broj čvorova: 40
  Broj grana: 224
vns_algorithm: broj cvorova na pocetku: 40
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 40
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 40
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 40
local_search: Broj cvorova na izlasku iz local_search 28 Broj grana na podgrafa 118
vns_algorithm: nadjeno je novo najbolje resenje: Nodes: 28 Edges:  118
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 24
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 24
local_search: Broj cvorova na izlasku iz local_search 24 Broj grana na podgrafa 78
vns_algorithm: nadjeno je novo najbolje resenje: Nodes: 24 Edges:  78
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 20
shaking: broj cvorova posle pomeranja cv

local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 25
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 25
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 25
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 62
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 21
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 18
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 18
local_search: Broj cvorova na izlasku iz local_search 15 Broj grana na podgrafa 38
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 21
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 21
local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 54
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 19
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 19
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 19 Broj grana na podgrafa 59
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 19 Broj grana na podgrafa 65
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 19
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 15
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 15
local_search: Broj cvorova na izlasku iz local_search 11 Broj grana na podgrafa 26
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 20
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 20
local_search: Broj cvorova na i

...

local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 53
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na izlasku iz local_search 17 Broj grana na podgrafa 53
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 53
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 21
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 21
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 21
local_search: Broj cvorova na izlasku iz local_search 17 Broj grana na podgrafa 51
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 20
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 16
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 16
local_search: Broj cvorova na i

shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 21
local_search: Broj cvorova na izlasku iz local_search 20 Broj grana na podgrafa 71
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 54
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 8
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 21
shaking: modifikovano resenje j

local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 50
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na izlasku iz local_search 22 Broj grana na podgrafa 75
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 19
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 16
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 16
local_search: Broj cvorova na izlasku iz local_search 16 Broj grana na podgrafa 46
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 20
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 20
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 19 Broj grana na podgrafa 57
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 21
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na i

In [19]:
ispisi_informacije_o_grafu(2)

Informacije o grafu 2:
  Broj čvorova: 45
  Broj grana: 290
vns_algorithm: broj cvorova na pocetku: 45
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 41
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 41
local_search: Broj cvorova na izlasku iz local_search 30 Broj grana na podgrafa 137
vns_algorithm: nadjeno je novo najbolje resenje: Nodes: 30 Edges:  137
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 26
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 25
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 25
local_search: Broj cvorova na izlasku iz local_search 24 Broj grana na podgrafa 95
vns_algorithm: nadjeno je novo najbolje resenje: Nodes: 24 Edges:  95
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 20
shaking: broj cvorova posle pomeranja cv

local_search: Broj cvorova na izlasku iz local_search 20 Broj grana na podgrafa 65
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 24
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 24
local_search: Broj cvorova na izlasku iz local_search 22 Broj grana na podgrafa 73
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 24
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 24
local_search: Broj cvorova na izlasku iz local_search 23 Broj grana na podgrafa 84
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 17 Broj grana na podgrafa 50
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 21
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 21
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 22 Broj grana na podgrafa 73
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 23
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 23
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 15 Broj grana na podgrafa 38
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na i

...

local_search: Broj cvorova na izlasku iz local_search 17 Broj grana na podgrafa 53
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na izlasku iz local_search 20 Broj grana na podgrafa 66
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 16 Broj grana na podgrafa 52
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 25
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 25
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 25
local_search: Broj cvorova na izlasku iz local_search 20 Broj grana na podgrafa 61
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na izlasku iz local_search 18 Broj grana na podgrafa 61
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 8
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na i

shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 22
local_search: Broj cvorova na izlasku iz local_search 19 Broj grana na podgrafa 56
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 19
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 19
local_search: Broj cvorova na izlasku iz local_search 14 Broj grana na podgrafa 37
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 22
shaking: modifikovano resenje j

local_search: Broj cvorova na izlasku iz local_search 17 Broj grana na podgrafa 43
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 7
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na izlasku iz local_search 10 Broj grana na podgrafa 23
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 23
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 23
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 23
local_search: Broj cvorova na izlasku iz local_search 23 Broj grana na podgrafa 75
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 25
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 24
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 24
local_search: Broj cvorova na i

local_search: Broj cvorova na izlasku iz local_search 21 Broj grana na podgrafa 68
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 23
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 23
local_search: Broj cvorova na izlasku iz local_search 21 Broj grana na podgrafa 74
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 24
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 24
shaking: modifikovano resenje je prihvaljivo
local_search: Broj cvorova na ulasku u local_search 24
local_search: Broj cvorova na izlasku iz local_search 19 Broj grana na podgrafa 64
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 6
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
local_search: Broj cvorova na ulasku u local_search 10
local_search: Broj cvorova na i